In [2]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

ERROR: Could not find a version that satisfies the requirement statsmodel==0.14.4 (from versions: none)
ERROR: No matching distribution found for statsmodel==0.14.4


In [3]:
import pandas as pd
import requests as rq
import lxml as lxml
from bs4 import BeautifulSoup
import io as io
import math
import gzip
import shutil
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import json
from pandasgui import show
import numpy as np
from io import BytesIO
from folium.plugins import HeatMap
import nbconvert

from script import request_tri
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [4]:
# Charger le fichier CSV contenant les transactions
transactions = pd.read_csv("data/base.csv",sep=";",encoding="utf-8")

C:\Users\flori\AppData\Local\Temp\ipykernel_26280\2978799986.py:2: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions = pd.read_csv("data/base.csv",sep=";",encoding="utf-8")


In [5]:
show(transactions.head(100))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\flori\AppData\Local\Temp\ipykernel_26280\1073385779.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  show(transactions.head(100))
C:\Users\flori\AppData\Local\Temp\ipykernel_26280\1073385779.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  show(transactions.head(100))
C:\Users\flori\AppData\Local\Temp\ipykernel_26280\1073385779.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.

In [6]:
# Ajouter une colonne pour savoir si le bien est en zone inondable
transactions[['results','identifiant_tri','libelle_type_inondation','code_scenario']] = transactions.apply(
    lambda row: pd.Series(request_tri.check_inondable(row['latitude'], row['longitude'])),
    axis=1
)


Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur A

In [7]:
# Sauvegarder le fichier avec les résultats

transactions['results'] = transactions['results'].replace({2: 0, 3: 0})
transactions = transactions.rename(columns={'results': 'zone_inondable'})
transactions.to_csv("transactions_with_zone_inondable.csv",sep=";",encoding="utf-8", index=False)

In [8]:
# import swifter

# test=transactions.iloc[0:1000]
# test[['results', 'identifiant_tri', 'libelle_type_inondation', 'code_scenario']] = test.swifter.apply(
#     lambda row: pd.Series(request_tri.check_inondable(row['latitude'], row['longitude'])),
#     axis=1
#     )

In [9]:
# Étape 1 : Compter et afficher le nombre de transactions par commune
transaction_counts = transactions.groupby('nom_commune').size().sort_values(ascending=False)
print(transaction_counts)

nom_commune
Nice                     7718
Cannes                   2772
Toulon                   2684
Saint-Raphaël            2288
Brest                    2026
                         ... 
Beauvoir-sur-Mer            1
Sainte-Marie-de-Ré          1
Lampaul-Plouarzel           1
Bourcefranc-le-Chapus       1
Hillion                     1
Length: 337, dtype: int64


In [10]:
filtered_transactions = transactions.groupby('nom_commune').filter(
    lambda group: len(group) >= 10 and group['results'].sum() > 0 and (group['results'] == 0).any()
)

KeyError: 'results'

In [ ]:
# Étape 3 : Calculer la moyenne agrégée des prix par commune et par zone inondable
filtered_transactions['prix_m2'] = filtered_transactions['valeur_fonciere'] / filtered_transactions['surface_reelle_bati']

In [ ]:
# Moyenne par commune et zone inondable
mean_prices = filtered_transactions.groupby(['nom_commune', 'results'])['prix_m2'].mean().unstack()

In [ ]:
# Calcul du prix moyen total par commune
filtered_transactions['prix_m2'] = filtered_transactions['valeur_fonciere'] / filtered_transactions['surface_reelle_bati']

# Calcul du prix moyen total par commune
mean_total = transactions.groupby('nom_commune')['prix_m2'].mean()

# Calcul du prix moyen par commune et par zone inondable
mean_by_zone = filtered_transactions.groupby(['nom_commune', 'results'])['prix_m2'].mean().unstack()

# Ajouter les colonnes pour le prix moyen en zone non inondable (results = 0) et zone inondable (results = 1)
mean_by_zone = mean_by_zone.rename(columns={0: 'prix_moyen_non_inondable', 1: 'prix_moyen_inondable'})

# Ajouter la colonne pour les prix moyens totaux
mean_by_zone['prix_moyen_total'] = mean_total

# Calculer la différence entre les prix moyens
mean_by_zone['différence_inondable'] = mean_by_zone['prix_moyen_inondable'] - mean_by_zone['prix_moyen_non_inondable']

# Ajouter le nombre d'observations en zone inondable et non inondable
observations_count = filtered_transactions.groupby(['nom_commune', 'results']).size().unstack(fill_value=0)
observations_count = observations_count.rename(columns={0: 'nb_non_inondable', 1: 'nb_inondable'})

# Joindre ces informations au DataFrame 'mean_by_zone'
mean_by_zone = mean_by_zone.join(observations_count)

# Trier les communes par différence décroissante
result_table = mean_by_zone.sort_values(by='différence_inondable', ascending=False)

# Résultat final
print(result_table)
print(result_table.dtypes)

# Exporter le DataFrame final en CSV
result_table.to_csv('data/moyenne.csv', encoding="utf-8", sep=";", decimal=",", index=True)

NameError: name 'filtered_transactions' is not defined